In [1]:
import hls4ml

#Data Processing
import numpy as np
from numpy import loadtxt, expand_dims
import h5py
import sys

#ML model
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense,Flatten, Dropout, Activation, concatenate, BatchNormalization, GRU, Add, Conv1D, Conv2D, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model


from sklearn.metrics import roc_curve
from sklearn.metrics import auc

#Plot settings
%matplotlib inline
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.ROOT)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 5

#Just to efficiently save plots
plot_dir = "../plot/"
print("Saving plot to: ", plot_dir)
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y")
save_path = plot_dir + dt_string

Saving plot to:  ../plot/


In [2]:
X_dummy = np.random.rand(10,9,1)

model = Sequential()
model.add(Conv1D(filters=5, kernel_size=3, strides=3, activation='relu',input_shape=X_dummy.shape[1:]))
model.add(Conv1D(filters=5, kernel_size=1,activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['binary_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 3, 5)              20        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 3, 5)              30        
_________________________________________________________________
flatten (Flatten)            (None, 15)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                160       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 271
Trainable params: 271
Non-trainable params: 0
__________________________________________________________

## 1. Convert the model into firmware

In [3]:
keras_model = model

# Create conversion config
yaml_config = {}
yaml_config['KerasModel'] = keras_model
yaml_config['OutputDir'] = 'L1B_toy'
yaml_config['ProjectName'] = 'myproject'
yaml_config['XilinxPart'] = 'xcvu9p-flgb2104-2-i'
yaml_config['ClockPeriod'] = 5
yaml_config['IOType'] = 'io_stream'
yaml_config['Backend'] = 'Vivado'

yaml_config['HLSConfig'] = {
'Model': {
'Precision': 'ap_fixed<16,8>',
'ReuseFactor': 10,
'Strategy': 'Resource',
'ConvImplementation': 'encoded'
}}


hls_model = hls4ml.converters.keras_to_hls(yaml_config)

Interpreting Sequential
Topology:
Layer name: conv1d_input, layer type: InputLayer, current shape: [[None, 9, 1]]
Layer name: conv1d, layer type: Conv1D, current shape: [[None, 9, 1]]
Layer name: conv1d_1, layer type: Conv1D, current shape: [[None, 3, 5]]
Layer name: dense, layer type: Dense, current shape: [[None, 3, 5]]
Layer name: dense_1, layer type: Dense, current shape: [[None, 10]]
Layer name: dense_2, layer type: Dense, current shape: [[None, 5]]
Creating HLS model


In [4]:
hls_model.compile()

Writing HLS project
Done


## 2. Make the ROC curve

## 3. build the model

In [5]:
hls_model.build(csim=False, reset = True)

{'EstimatedClockPeriod': '4.361',
 'BestLatency': '163',
 'WorstLatency': '180',
 'IntervalMin': '25',
 'IntervalMax': '181',
 'BRAM_18K': '15',
 'DSP48E': '42',
 'FF': '3300',
 'LUT': '7380',
 'URAM': '0',
 'AvailableBRAM_18K': '4320',
 'AvailableDSP48E': '6840',
 'AvailableFF': '2364480',
 'AvailableLUT': '1182240',
 'AvailableURAM': '960'}

In [7]:
hls4ml.report.read_vivado_report('L1B_toy')

Found 1 solution(s) in L1B_toy/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Mon May 24 13:53:08 2021

* Version:        2019.2.1 (Build 2724168 on Thu Dec 05 05:19:09 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcvu9p-flgb2104-2-i


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 5.00 ns | 4.361 ns |   0.62 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---------+----------+---------

-----